# Задание взято с [платформы](https://karpov.courses/)
## Домашнее задание
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили [данные](https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q) из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

### Описание данных 

`date` – дата совершения события

`event` - событие

- app_install – установка приложения

- app_start – открыть приложения

- register – зарегистрироваться 

- search – перейти на страницу поиска товаров (каталог)

- choose_item – отправить товар в корзину

- tap_basket – перейти в корзину

- purchase – подтверждение покупки

`gender` – пол пользователя

`os_name` – платформа пользователя

`city` – город пользователя

`device_id` – идентификатор устройства пользователя

`urm_source` – канал, с которого пришел пользователь

- yandex-direct – Яндекс директ

- google_ads – реклама в Google

- vk_ads – реклама в ВК

- instagram_ads – реклама в instagram

- facebook_ads – реклама в facebook

- referal – акция «приведи друга»

- Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  

`purchase_sum` – стоимость покупки (при совершении события `purchase`)

### Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from urllib.parse import urlencode
import json
import re

sns.set(rc={'figure.figsize':(12,6)}, style="whitegrid")

In [2]:
def parse_data(base_url, public_key):
    
    final_url = base_url + urlencode(dict(public_key=public_key))
    response = requests.get(final_url)
    download_url = response.json()['href']
    
    return pd.read_csv(download_url)

In [3]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
data_url = 'https://disk.yandex.ru/d/p_UjGG_Dv8CE5Q'
df = parse_data(base_url, data_url)
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [4]:
df.dtypes

date             object
event            object
purchase_sum    float64
os_name          object
device_id         int64
gender           object
city             object
utm_source       object
dtype: object

MAU февраля:

In [5]:
df.query('(date < "2020-03-01") & (date >= "2020-02-01")').device_id.nunique()

75032

Количество установок в январе:

In [6]:
len(df.query('(date < "2020-02-01") & (event == "app_install") '))

80297

Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

In [7]:
df_installs = df.query('event == "app_install"') 

In [8]:
df_installs.groupby('device_id', as_index=False) \
    .agg({'date': 'count'}).date.unique()

array([1])

In [9]:
df_purchases = df.query('event == "purchase"').groupby('device_id', as_index = False) \
    .agg({'date': 'min'}) \
    .rename(columns={'date': 'purchase_date'})

In [10]:
df_installs_purchases = pd.merge(df_installs, df_purchases, how='left', on='device_id')

In [11]:
df_installs_purchases['DaysPurchase'] = (pd.to_datetime(df_installs_purchases.purchase_date) - pd.to_datetime(df_installs_purchases.date)) <= np.timedelta64 (7, 'D')


In [12]:
df_cr = pd.DataFrame()

df_cr = df_cr.assign(date=df_installs_purchases[df_installs_purchases.DaysPurchase].groupby('date', as_index=False) \
    .agg({'event': 'count'})['date'],
             cr=df_installs_purchases[df_installs_purchases.DaysPurchase].groupby('date', as_index=False) \
    .agg({'event': 'count'})['event']/df_installs_purchases.groupby('date', as_index=False) \
    .agg({'event': 'count'})['event'])

In [13]:
df_cr.sort_values('cr', ascending=False).head()

,date,cr
0,2020-01-01,0.393406
8,2020-01-09,0.391854
14,2020-01-15,0.382831
13,2020-01-14,0.381403
1,2020-01-02,0.377226


С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [14]:
marketing_df = df.groupby('device_id', as_index=False) \
    .agg({'utm_source': 'unique'}) 
marketing_df.explode('utm_source').value_counts('utm_source')

utm_source
-                52273
yandex-direct    40712
google_ads       38096
vk_ads           34086
instagram_ads    31048
facebook_ads     25959
referal          15926
dtype: int64

Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?


In [15]:
df = pd.merge(df, df[df.event == 'register'].groupby('device_id', as_index=False) \
    .agg({'date': 'min'}), how='left', on='device_id') \
    .rename(columns={
        'date_x': 'date',
        'date_y':'register_date'})


In [16]:
answer = pd.DataFrame()


answer.assign(event=df[df['register_date'].isna() == False].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 4, 5, 2, 6, 3]].event,
    cr=df[df['register_date'].isna() == False].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 4, 5, 2, 6, 3]].count_events/df[df['register_date'].isna() == False].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 4, 5, 2, 6, 3]].count_events.shift(1))

,event,cr
0,app_install,NaN
1,app_start,9.361868
4,register,0.131675
5,search,7.218376
2,choose_item,0.779214
6,tap_basket,0.731694
3,purchase,0.438687


In [17]:
answer = pd.DataFrame()


answer.assign(event=df[df['register_date'].isna()].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 3, 2, 4]].event,
    cr=df[df['register_date'].isna()].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 3, 2, 4]].count_events/df[df['register_date'].isna()].groupby('event') \
    .agg({'event': 'count'}) \
    .rename(columns={'event': 'count_events'}) \
    .reset_index().iloc[[0, 1, 3, 2, 4]].count_events.shift(1))

,event,cr
0,app_install,NaN
1,app_start,1.690802
3,search,0.931064
2,choose_item,0.684965
4,tap_basket,0.563919


Пользователи, пришедшие с какого канала, показали самую низкую конверсию в первую покупку?

CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [47]:
cr_source = pd.merge(df, df.query('(event == "app_start") | (event == "purchase")') \
    .groupby(['device_id', 'event'], as_index=False) \
    .agg({'date': 'min'}), how='inner', on=['device_id', 'date', 'event'] )

cr_source= cr_source.groupby(['utm_source', 'event'], as_index=False) \
    .agg({'date': 'count'})


cr_source = cr_source.pivot(index='utm_source', columns='event', values='date')

cr_source['CR'] = cr_source['purchase']/cr_source['app_start']

array(['app_start', 'purchase'], dtype=object)

In [52]:
cr_source

event,app_start,purchase,CR
utm_source,,,
-,41456,14786,0.356667
facebook_ads,18844,7903,0.419391
google_ads,31437,10167,0.323409
instagram_ads,24818,9820,0.395681
referal,11983,5803,0.484269
vk_ads,27905,11460,0.410679
yandex-direct,34441,10936,0.317529


Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

In [43]:
pd.merge(df, df.query('event == "purchase"') \
    .groupby(['device_id'], as_index=False) \
    .agg({'date': 'min'}), how='inner', on=['device_id', 'date'] ).dropna() \
    .groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'median'}) \
    .sort_values('purchase_sum', ascending=False)




,utm_source,purchase_sum
0,-,398.5
4,referal,395.5
3,instagram_ads,393.5
5,vk_ads,393.0
6,yandex-direct,392.5
2,google_ads,390.5
1,facebook_ads,389.0


Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI? 

In [65]:
sources = list(df.groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'sum'}).iloc[[6,2,1,3,5]].utm_source)

revenues = list(df.groupby('utm_source', as_index=False) \
    .agg({'purchase_sum': 'sum'}).iloc[[6,2,1,3,5]].purchase_sum)

expenses = [10491707, 10534878, 8590498, 8561626, 9553531]

for i in range(len(sources)):
    ROMI = (revenues[i]-expenses[i])/expenses[i]
    print(sources[i], "ROMI: ", ROMI)

yandex-direct ROMI:  0.3263206835646478
google_ads ROMI:  0.22149264566708793
facebook_ads ROMI:  0.4259826380263403
instagram_ads ROMI:  0.6990895187432854
vk_ads ROMI:  0.7155596710786828
